In [0]:
path = '/content/gdrive/My Drive/Demo ML/Devoir4/'

In [4]:
import numpy as np 
import matplotlib.pyplot as plt
%pylab inline

#Link to my google drive
from google.colab import drive
drive.mount('/content/gdrive')

Populating the interactive namespace from numpy and matplotlib
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Load images with numpy
images_train = np.load(path + 'train_images.npy', encoding='latin1')
images_train = images_train[:,1]/255

#Load labels
train_labels = np.genfromtxt(path + 'train_labels.csv', names=True, delimiter=',', dtype=[('Id', 'i8'), ('Category', 'U15')])

#Load images to predict
images_test = np.load(path + 'test_images.npy', encoding='latin1')
images_test = images_test[:,1]


In [0]:
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

labEncoder = LabelEncoder()
labEncoder.fit(train_labels['Category'])
label = labEncoder.transform(train_labels['Category'])

# Transformer les catégories en codage OneHot
# oneHot = OneHotEncoder(n_values=num_classes, sparse=False)
# labelOneHot = oneHot.fit_transform(label.reshape(-1,1))
# oneHot.fit(label.reshape(-1,1))
# labelOneHot = oneHot.transform(label.reshape(-1,1))


In [0]:
#Preprocessing functions

class block:
  
    def __init__(self, d, x, y):
      queue = [(x,y)]
      self.visited = []
      est = x
      west = x
      north = y
      south = y
      flag = np.zeros((d.shape[0],d.shape[1]))

      while queue:
        
#           print("queue is", queue)
          s = queue.pop(0)
          self.visited.append(s)
          flag[s[0]][s[1]] = 1
        
#         if not flag[s[0]][s[1]]:
          

          if s[0]+1 < 100 and d[s[0]+1][s[1]] > 0 and not flag[s[0]+1][s[1]]:
            queue.append((s[0]+1,s[1]))
            flag[s[0]+1][s[1]] = 1
            if s[0]+1 > west:
              west = s[0]+1

          if s[1]+1 < 100 and d[s[0]][s[1]+1] > 0 and not flag[s[0]][s[1]+1]:
            queue.append((s[0],s[1]+1))
            flag[s[0]][s[1]+1] = 1
            if s[1]+1 > south:
              south = s[1]+1

          if s[0]-1 > 0 and d[s[0]-1][s[1]] > 0 and not flag[s[0]-1][s[1]]:
            queue.append((s[0]-1,s[1]))
            flag[s[0]-1][s[1]] = 1
            if s[0]-1 < est:
              est = s[0]-1

          if s[1]-1 > 0 and d[s[0]][s[1]-1] > 0 and not flag[s[0]][s[1]-1]:
            queue.append((s[0],s[1]-1))
            flag[s[0]][s[1]-1] = 1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]+1 < 100 and s[1]+1 < 100 and d[s[0]+1][s[1]+1] > 0 and not flag[s[0]+1][s[1]+1]:
            queue.append((s[0]+1,s[1]+1))
            flag[s[0]+1][s[1]+1] = 1
            if s[0]+1 > west:
              west = s[0]+1
            if s[1]+1 > south:
              south = s[1]+1

          if s[0]-1 > 0 and s[1]-1 > 0 and d[s[0]-1][s[1]-1] > 0 and not flag[s[0]-1][s[1]-1]:
            queue.append((s[0]-1,s[1]-1))
            flag[s[0]-1][s[1]-1] = 1
            if s[0]-1 < est:
              est = s[0]-1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]+1 < 100 and s[1]-1 > 0 and d[s[0]+1][s[1]-1] > 0 and not flag[s[0]+1][s[1]-1]:
            queue.append((s[0]+1,s[1]-1))
            flag[s[0]+1][s[1]-1] = 1
            if s[0]+1 > west:
              west = s[0]+1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]-1 > 0 and s[1]+1 < 100 and d[s[0]-1][s[1]+1] > 0 and not flag[s[0]-1][s[1]+1]:
            queue.append((s[0]-1,s[1]+1))
            flag[s[0]-1][s[1]+1] = 1
            if s[0]-1 < est:
              est = s[0]-1
            if s[1]+1 > south:
              south = s[1]+1

      self.size = (west-est+1, south-north+1)

def clean_noise(d, noise_size):
    negative = []
    for i in range(d.shape[0]):
      for j in range(d.shape[1]):
        if d[i][j] > 0:
          b = block(d,i,j)
          if b.size < (noise_size[0],noise_size[1]):
            for dots in b.visited:
              d[dots[0]][dots[1]] = 0 
          else:
            negative.append(b.visited)
            for dots in b.visited:
              d[dots[0]][dots[1]] = -d[dots[0]][dots[1]]
    
    biggest = 0
    for h in negative:
      if len(h) > biggest:
        biggest = len(h)
    
    for g in negative:
        if len(g) == biggest:
          for p in g:
            d[p[0]][p[1]] = -d[p[0]][p[1]]
        else:
          for p in g:
            d[p[0]][p[1]] = 0
    
    return d

# def clean_noise(d, noise_size):
#     negative = []
#     for i in range(d.shape[0]):
#       for j in range(d.shape[1]):
# #         print("d[",i,"][",j,"] ",d[i][j])
#         if d[i][j] > 0:
#           b = block(d,i,j)
#           if b.size < (noise_size[0],noise_size[1]):
#             for dots in b.visited:
#               d[dots[0]][dots[1]] = 0
# #             print("cleaned a noise ...")  
#           else:
#             negative.append(b.visited)
#             for dots in b.visited:
#               d[dots[0]][dots[1]] = -d[dots[0]][dots[1]]
    
#     for k in negative:
#       for g in k:
#         d[g[0]][g[1]] = -d[g[0]][g[1]]
    
#     return d

#######################################################

def centralize(image, dimInitial, dimCentre):
#Find bounding box of the image
  col_sum = np.where(np.sum(image, axis = 0)>0)
  row_sum = np.where(np.sum(image, axis = 1)>0)
  
  if len(col_sum[0])== 0 and len(row_sum[0]) == 0 :
    return image[0:dimCentre, 0:dimCentre]
  
  y1 = row_sum[0][0]
  x1 = col_sum[0][0]
  
  padX, padY = 0, 0
  y2 = y1 + dimCentre
  x2 = x1 + dimCentre
  
  if x2 > dimInitial:
    padX = x2 - dimInitial
    x2 = dimInitial
    
  if y2 > dimInitial:
    padY = y2 - dimInitial
    y2 = dimInitial
    
  img = image[y1:y2, x1:x2]
  
  if padX > 0:
    pad = np.zeros((y2-y1, padX))
    img = np.concatenate((img, pad), axis = 1)
  
  if padY > 0:
    pad = np.zeros((padY, dimCentre))
    img = np.concatenate((img, pad), axis = 0)
    
  return img

# image = images_train[44].reshape(100,100)
# print("Before noise reduction")
# plt.imshow(image)
# plt.show()
# # for i in range(image.shape[0]):
# #   for j in range(image.shape[1]):
# #     if image[i][j]<0:
# #       print(image[i][j])

# clean_noise(image,noise_size=(20,20))

# print("After noise reduction")
# plt.imshow(image)
# plt.show()

In [0]:
#Creer des images transformé de images train
import cv2 as cv

def translation(img, x, y):
  rows,cols = img.shape
  M = np.float32([[1,0,x],[0,1,y]])
  return cv.warpAffine(img,M,(cols,rows))

def rotation(img, angle):
  rows,cols = img.shape
  # cols-1 and rows-1 are the coordinate limits.
  M = cv.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0),angle,1)
  return cv.warpAffine(img,M,(cols,rows))

def transformerDataset(dataSet, d = 32):
  
  n = dataSet.shape[0]
  
  trainTransforme = np.zeros((n, d*d))
  
  for i in range(n//4):
    img = dataSet[i].reshape(d,d)
    img = translation(img,3,3) 
    trainTransforme[i] = img.reshape(1, d*d)
  
  for i in range(n//4, 2*(n//4)):
    img = dataSet[i].reshape(d,d)
    img = translation(img,3,3)
    img = rotation(img,20) 
    trainTransforme[i] = img.reshape(1, d*d)

  for i in range(2*(n//4), 3*(n//4)):
    img = dataSet[i].reshape(d,d)
    img = rotation(img,340) 
    trainTransforme[i] = img.reshape(1, d*d)
    
  for i in range(3*(n//4), n):
    img = dataSet[i].reshape(d,d)
    img = rotation(img,320) 
    trainTransforme[i] = img.reshape(1, d*d)
    
  return trainTransforme






In [0]:
# # chargment de données
# trainSet = np.zeros((num_train, num_features))
# for i in range(num_train):
#     trainSet[i] = images_train[i].tolist()

from skimage.transform import rescale
num_train = images_train.shape[0]
num_classes = 31
dimCentre = 32
num_features = dimCentre*dimCentre


#Nettoyer les bruits
trainProcessed = np.zeros((num_train, num_features))

for i in range(num_train):
  image = images_train[i].reshape(100,100)
  clean_noise(image,noise_size=(9,9))
  image = centralize(image, 100, dimCentre)
  trainProcessed[i] = image.reshape(1, num_features)

#Crees les images transformer
imTrans = transformerDataset(trainProcessed)

#Joindre les 2 dataset
newTrain = np.concatenate((trainProcessed, imTrans), axis=0)  
newLabel = np.concatenate((label, label), axis=0)

In [0]:

# Setup
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag

platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

import torch
device = torch.device('cuda:0')



In [0]:
### CODE MODIFIED FROM PYTORCH EXAMPLE REPO ###

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

torch.manual_seed(0)
torch.cuda.manual_seed(0)

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

flatten = Flatten()

def train(model, device, train, target, optimizer, epoch, loss_fn, nbOfBatches):
  model.train()
  
  trainBatches = np.split(train, nbOfBatches)
  labelBatches = np.split(target, nbOfBatches)
  
  for i in range(nbOfBatches):
 
      data = torch.from_numpy(trainBatches[i]).float().to(device)
      target = torch.from_numpy(labelBatches[i]).long().to(device)
      optimizer.zero_grad()
      output = model(flatten(data))
      loss = loss_fn(output, target)
      loss.backward()
      optimizer.step()
      if i%10 == 0:
        print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, i * len(data), len(train),
                loss.item()))


    
def valider(model, device, test, target, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        test = torch.from_numpy(test).float().to(device)
        target = torch.from_numpy(target).long().to(device)
        output = model(flatten(test))
        test_loss += loss_fn(output, target).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    print("Validation loss: ", test_loss / len(test), "Accuracy: ", correct/len(test))
    return test_loss / len(test), correct/len(test)
    
    


In [62]:
optimizer = optim.Adam(model.parameters())
#loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')
# test_loss_fn = nn.CrossEntropyLoss()

# 2n best
model = torch.nn.Sequential(
    torch.nn.Linear(num_features, 800),
    torch.nn.ReLU(),
    torch.nn.Linear(800, 600),
    torch.nn.ReLU(),
    torch.nn.Linear(600, 200),
    torch.nn.ReLU(),
    torch.nn.Linear(200, 80),
    torch.nn.ReLU(),
    torch.nn.Linear(80, num_classes)
).to(device)

optimizer = optim.Adam(model.parameters())

epoch=20
nbOfBatches = 50

#Cross validation
#Shuffle data
randomIdx = np.arange(num_train)
np.random.seed(500)
np.random.shuffle(randomIdx)
trainSetRand = trainProcessed[randomIdx]
labelRand = label[randomIdx]

k = 10
kTrain = np.split(trainSetRand, k)
kLabel = np.split(labelRand, k) 

losses = []
accuracies = []
# for e in range(epoch):
for i in range(k):
  for j in range(k):
    if j != i:
      train(model, device, kTrain[j], kLabel[j], optimizer, 1, loss_fn, nbOfBatches)

  loss, accuracy = valider(model, device, kTrain[i], kLabel[i], test_loss_fn)
  losses.append(loss)
  accuracies.append(accuracy)
  
print("Performance avec 4 couches cachées")
print("Loss moyenne: ", np.mean(losses), "Accuracy moyenne: ", np.mean(accuracy))

Train Epoch: 1 [0/1000]	Loss: 3.427819
Train Epoch: 1 [200/1000]	Loss: 3.469441
Train Epoch: 1 [400/1000]	Loss: 3.397414
Train Epoch: 1 [600/1000]	Loss: 3.296280
Train Epoch: 1 [800/1000]	Loss: 3.457532
Train Epoch: 1 [0/1000]	Loss: 2.931284
Train Epoch: 1 [200/1000]	Loss: 3.458836
Train Epoch: 1 [400/1000]	Loss: 3.040128
Train Epoch: 1 [600/1000]	Loss: 3.227544
Train Epoch: 1 [800/1000]	Loss: 2.812342
Train Epoch: 1 [0/1000]	Loss: 2.507524
Train Epoch: 1 [200/1000]	Loss: 2.945917
Train Epoch: 1 [400/1000]	Loss: 2.530832
Train Epoch: 1 [600/1000]	Loss: 2.937170
Train Epoch: 1 [800/1000]	Loss: 2.760964
Train Epoch: 1 [0/1000]	Loss: 2.507678
Train Epoch: 1 [200/1000]	Loss: 2.676030
Train Epoch: 1 [400/1000]	Loss: 2.821646
Train Epoch: 1 [600/1000]	Loss: 2.698173
Train Epoch: 1 [800/1000]	Loss: 2.825157
Train Epoch: 1 [0/1000]	Loss: 2.593435
Train Epoch: 1 [200/1000]	Loss: 2.295002
Train Epoch: 1 [400/1000]	Loss: 2.698282
Train Epoch: 1 [600/1000]	Loss: 2.337677
Train Epoch: 1 [800/1000]	

In [63]:
optimizer = optim.Adam(model.parameters())

loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')
# test_loss_fn = nn.CrossEntropyLoss()
model = torch.nn.Sequential( 
    torch.nn.Linear(num_features, 700),
    torch.nn.ReLU(),
    torch.nn.Linear(700, 200),
    torch.nn.ReLU(),
    torch.nn.Linear(200, 80),
    torch.nn.ReLU(),
    torch.nn.Linear(80, num_classes)
).to(device)

optimizer = optim.Adam(model.parameters())

epoch=20
nbOfBatches = 50

#Cross validation
#Shuffle data
randomIdx = np.arange(num_train)
np.random.seed(500)
np.random.shuffle(randomIdx)
trainSetRand = trainProcessed[randomIdx]
labelRand = label[randomIdx]

k = 10
kTrain = np.split(trainSetRand, k)
kLabel = np.split(labelRand, k) 

losses = []
# for e in range(epoch):
for i in range(k):
  for j in range(k):
    if j != i:
      train(model, device, kTrain[j], kLabel[j], optimizer, 1, loss_fn, nbOfBatches)

    loss, accuracy = valider(model, device, kTrain[i], kLabel[i], test_loss_fn)
    losses.append(loss)
    accuracies.append(accuracy)


print("Performance avec 3 couches cachées")
print("Loss moyenne: ", np.mean(losses), "Accuracy moyenne: ", np.mean(accuracy))

Validation loss:  3.436209228515625 Accuracy:  0.03
Train Epoch: 1 [0/1000]	Loss: 3.399767
Train Epoch: 1 [200/1000]	Loss: 3.391805
Train Epoch: 1 [400/1000]	Loss: 3.390975
Train Epoch: 1 [600/1000]	Loss: 3.231767
Train Epoch: 1 [800/1000]	Loss: 3.164120
Validation loss:  2.880922119140625 Accuracy:  0.208
Train Epoch: 1 [0/1000]	Loss: 2.697506
Train Epoch: 1 [200/1000]	Loss: 3.152102
Train Epoch: 1 [400/1000]	Loss: 2.497268
Train Epoch: 1 [600/1000]	Loss: 3.070365
Train Epoch: 1 [800/1000]	Loss: 2.496755
Validation loss:  2.62900927734375 Accuracy:  0.236
Train Epoch: 1 [0/1000]	Loss: 2.473528
Train Epoch: 1 [200/1000]	Loss: 2.562886
Train Epoch: 1 [400/1000]	Loss: 2.377258
Train Epoch: 1 [600/1000]	Loss: 3.037191
Train Epoch: 1 [800/1000]	Loss: 2.572754
Validation loss:  2.5230244140625 Accuracy:  0.26
Train Epoch: 1 [0/1000]	Loss: 2.319668
Train Epoch: 1 [200/1000]	Loss: 2.710576
Train Epoch: 1 [400/1000]	Loss: 2.649977
Train Epoch: 1 [600/1000]	Loss: 2.521021
Train Epoch: 1 [800/10

In [64]:
optimizer = optim.Adam(model.parameters())

loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')
# test_loss_fn = nn.CrossEntropyLoss()
model = torch.nn.Sequential( 
    torch.nn.Linear(num_features, 500),
    torch.nn.ReLU(),
    torch.nn.Linear(500, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, num_classes)
).to(device)

optimizer = optim.Adam(model.parameters())

epoch=20
nbOfBatches = 50

#Cross validation
#Shuffle data
randomIdx = np.arange(num_train)
np.random.seed(500)
np.random.shuffle(randomIdx)
trainSetRand = trainProcessed[randomIdx]
labelRand = label[randomIdx]

k = 10
kTrain = np.split(trainSetRand, k)
kLabel = np.split(labelRand, k) 

losses = []
# for e in range(epoch):
for i in range(k):
  for j in range(k):
    if j != i:
      train(model, device, kTrain[j], kLabel[j], optimizer, 1, loss_fn, nbOfBatches)

    loss, accuracy = valider(model, device, kTrain[i], kLabel[i], test_loss_fn)
    losses.append(loss)
    accuracies.append(accuracy)

    
print("Performance avec 2 couches cachées")
print("Loss moyenne: ", np.mean(losses), "Accuracy moyenne: ", np.mean(accuracy))

Validation loss:  3.4385068359375 Accuracy:  0.035
Train Epoch: 1 [0/1000]	Loss: 3.408541
Train Epoch: 1 [200/1000]	Loss: 3.346329
Train Epoch: 1 [400/1000]	Loss: 3.279877
Train Epoch: 1 [600/1000]	Loss: 3.212991
Train Epoch: 1 [800/1000]	Loss: 3.158059
Validation loss:  2.855260009765625 Accuracy:  0.218
Train Epoch: 1 [0/1000]	Loss: 2.674887
Train Epoch: 1 [200/1000]	Loss: 3.153062
Train Epoch: 1 [400/1000]	Loss: 2.479416
Train Epoch: 1 [600/1000]	Loss: 3.005108
Train Epoch: 1 [800/1000]	Loss: 2.426524
Validation loss:  2.61614111328125 Accuracy:  0.231
Train Epoch: 1 [0/1000]	Loss: 2.460289
Train Epoch: 1 [200/1000]	Loss: 2.399251
Train Epoch: 1 [400/1000]	Loss: 2.314442
Train Epoch: 1 [600/1000]	Loss: 3.002831
Train Epoch: 1 [800/1000]	Loss: 2.407075
Validation loss:  2.47218701171875 Accuracy:  0.299
Train Epoch: 1 [0/1000]	Loss: 2.313367
Train Epoch: 1 [200/1000]	Loss: 2.540901
Train Epoch: 1 [400/1000]	Loss: 2.742525
Train Epoch: 1 [600/1000]	Loss: 2.519984
Train Epoch: 1 [800/1

In [0]:
# HYPERPARAMÈTRES
    
device = torch.device("cuda")
# device = torch.device("cpu")


# best
# model = torch.nn.Sequential( 
#     torch.nn.Linear(num_features, 700),
#     torch.nn.ReLU(),
#     torch.nn.Linear(700, 200),
#     torch.nn.ReLU(),
#     torch.nn.Linear(200, 80),
#     torch.nn.ReLU(),
#     torch.nn.Linear(80, num_classes)
# ).to(device)

model = torch.nn.Sequential( 
    torch.nn.Linear(num_features, 500),
    torch.nn.ReLU(),
    torch.nn.Linear(500, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, num_classes)
).to(device)


epochs1 = 150
# epochs2 = 150
# epochs3 = 100

# optimizer = optim.SGD(model.parameters(), lr=1)
# optimizer2 = optim.SGD(model.parameters(), lr=0.01)
# optimizer3 = optim.SGD(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters())
optimizer2 = optim.Adam(model.parameters(), lr=0.005)
optimizer3 = optim.Adam(model.parameters(), lr=0.001)



#loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')
# test_loss_fn = nn.CrossEntropyLoss()
  


In [68]:
#Shuffle data
randomIdx = np.arange(2*num_train)
np.random.seed(570)
np.random.shuffle(randomIdx)
trainSetRand = newTrain[randomIdx]
labelRand = newLabel[randomIdx]

# entrainment
for epoch in range(1, epochs1):
  train(model, device, trainSetRand[:9000], labelRand[:9000], optimizer, epoch, loss_fn, nbOfBatches)
  valider(model, device, trainSetRand[9000:], labelRand[9000:], test_loss_fn)
            
# for epoch in range(epochs1, epochs2):
#   train(model, device, trainSetRand[:9000], labelRand[:9000], optimizer2, epoch, loss_fn, nbOfBatches)
#   valider(model, device, trainSetRand[9000:], labelRand[9000:], test_loss_fn)
  
# for epoch in range(epochs2, epochs3):
#   train(model, device, trainSetRand[:9000], label[:9000], optimizer3, epoch, loss_fn, nbOfBatches)
#   valider(model, device, validBatches, labelTest, test_loss_fn)

Train Epoch: 1 [0/9000]	Loss: 3.437510
Train Epoch: 1 [1800/9000]	Loss: 3.210114
Train Epoch: 1 [3600/9000]	Loss: 3.065531
Train Epoch: 1 [5400/9000]	Loss: 2.962631
Train Epoch: 1 [7200/9000]	Loss: 2.681681
Validation loss:  2.723750177556818 Accuracy:  0.23836363636363636
Train Epoch: 2 [0/9000]	Loss: 2.608649
Train Epoch: 2 [1800/9000]	Loss: 2.627746
Train Epoch: 2 [3600/9000]	Loss: 2.542326
Train Epoch: 2 [5400/9000]	Loss: 2.552869
Train Epoch: 2 [7200/9000]	Loss: 2.375862
Validation loss:  2.504955255681818 Accuracy:  0.28809090909090906
Train Epoch: 3 [0/9000]	Loss: 2.291541
Train Epoch: 3 [1800/9000]	Loss: 2.389063
Train Epoch: 3 [3600/9000]	Loss: 2.301086
Train Epoch: 3 [5400/9000]	Loss: 2.338339
Train Epoch: 3 [7200/9000]	Loss: 2.162728
Validation loss:  2.3925397727272726 Accuracy:  0.31427272727272726
Train Epoch: 4 [0/9000]	Loss: 2.067449
Train Epoch: 4 [1800/9000]	Loss: 2.175250
Train Epoch: 4 [3600/9000]	Loss: 2.099157
Train Epoch: 4 [5400/9000]	Loss: 2.174628
Train Epoch:

In [0]:
num_test = images_test.shape[0]
# traitement des images à prédire
testSet = np.zeros((num_test, num_features))
for i in range(num_test):
  image = images_test[i].reshape(100,100)
  clean_noise(image,noise_size=(9,9))
  image = centralize(image, 100, dimCentre)
  testSet[i] = image.reshape(1, num_features)

testSet = torch.from_numpy(testSet).float().to(device) 

#Faire la prédiction
prediction = model(testSet)
prediction = prediction.max(1, keepdim=False)[1]



# for i in range(num_train):
#   image = images_train[i].reshape(100,100)
#   clean_noise(image,noise_size=(9,9))
#   image = centralize(image, 100, dimCentre)
#   trainProcessed[i] = image.reshape(1, num_features)

In [69]:

predLabel = labEncoder.inverse_transform(prediction)
# print(predLabel[0])

id = np.arange(num_test)
x = []
for i in range(num_test):
    x.append((id[i], predLabel[i]))


# Generate Submission File 
np.savetxt(path + 'prediction.csv', x, fmt='%s', delimiter=',', header="Id, Category")



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
